# Farm product prices, crops and livestock
https://open.canada.ca/data/en/dataset/d5614095-e77a-4cb4-a5e6-9f8bff067c9f
Number of records: 109952

### Possible questions:
- Which products have shown the greatest change in price
- How does production of a given product correlate with price? For example, if we looked at egg production in response to prices https://open.canada.ca/data/en/dataset/59a62325-9857-491e-8291-fe164c342e48/resource/e44d58af-e1fa-4719-966b-a69edcd4672d 


Loganberries, cranberries and blueberries, production and value
https://open.canada.ca/data/en/dataset/896db994-8209-4a94-b4d0-37d07e1d2698


Another idea was to explore the impact of Canadian crop prices on crop production. We would tackle this question by exploring datasets released by StatsCan / Agricultural Census. 

This dataset provides a time series of agricultural product prices within Canada. We would try to establish a relationship between the price of a crop and its production over time. As an example, if we decide to explore this relationship with table egg production, here is a dataset looking at egg yields over time.

We can also explore rates of change in pricing for the different product types over time. 

